In [1]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd

/home/yhchoi/SSPreprocess/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
all_clinical_features = pd.read_excel('/home/yhchoi/FSTrain_0205/XGBoost/features_0228/feautres.xlsx')

acfdict = dict(zip(all_clinical_features['Unnamed: 0'], all_clinical_features['Unnamed: 1']))
acfdict_bykey = {1:[], 2:[], 3:[], 4:[], 5:[], 6:[]}

for key, value in acfdict.items():
    acfdict_bykey[value].append(key)

acfdict_bykey[6] = [item for item in acfdict_bykey[6] if item not in ['serum_d_dimer', 'serum_iCa', 'serum_iMg', 'serum_CK_MB', 'serum_TG', 'serum_HDL', 'serum_amylase', 'serum_lipase', 'serum_TSH', 'serum_T3', 'serum_fT4', 'serum_vitB12', 'serum_delta_neutrophil']]
id_lookups = pd.read_csv('/home/yhchoi/FSTrain_0205/src/labels/id_lookups.csv')
prognosis_labels = pd.read_csv('/home/yhchoi/FSTrain_0205/src/labels/prognosis_labels_tidiedup.csv')
all_clinical = pd.read_csv('/home/yhchoi/FSTrain_0205/XGBoost/features_0228/patient_sinchon_230411.csv')
idlprog = pd.merge(id_lookups, prognosis_labels, on='ID')
idlprog = idlprog.drop(['final_pathogen', 'classification', 'pathogen', 'age', 'MRI', 'True_id'], axis=1)
clinicals = pd.merge(idlprog, all_clinical, on='ID')
clinicals = clinicals.drop(['True_id', 'pathogen_label2', 'ICU_duration', 'ICU_label', 'LOS', 'final_pathogen', 'classification', 'Confirm', 'pathogen_label', 'death_label', 'death_label2', 'serum_delta_neutrophil'], axis=1)
clinicals.fillna(-1, inplace=True)
clinicals.to_csv('/home/yhchoi/FSTrain_0205/XGBoost/features_0228/clinical_features.csv', index=False)
traindata_new = pd.read_excel('/home/yhchoi/FSTrain_0205/XGBoost/InnerBrain/train_data.xlsx')
traindata_new = traindata_new[['Filename', 'Prognosis', 'Pathogen']]
traindatasets = [pd.merge(traindata_new, clinicals[['Filename'] + acfdict_bykey[i]], on='Filename') for i in range(1, 7)]
testdata_new = pd.read_excel('/home/yhchoi/FSTrain_0205/XGBoost/InnerBrain/test_data.xlsx')
testdata_new = testdata_new[['Filename', 'Prognosis', 'Pathogen']]
testdatasets = [pd.merge(testdata_new, clinicals[['Filename'] + acfdict_bykey[i]], on='Filename') for i in range(1, 7)]

In [17]:
# pathogen = 1
# featnum = 5
# Select feature columns
for pathogen in [0, 1, 2, 3]:
    feature_columns = acfdict_bykey[1] + acfdict_bykey[3] + acfdict_bykey[4] + acfdict_bykey[5] + acfdict_bykey[6] + ['CXR_abnormal', 'CT_abnormal']
    filenames_internal_train = torch.load(f'/home/yhchoi/FSTrain_0205/samplevectors_internal/{pathogen}/filenames_train.pt')
    rows = []
    for filename in filenames_internal_train:
        # dataframe row to list
        row = clinicals.loc[clinicals['Filename'] == filename.replace(filename.split('_')[0] + '_' + filename.split('_')[1] + '_', '')]
        row = row[feature_columns].values.tolist()
        rows += row

    filenames_internal_test = torch.load(f'/home/yhchoi/FSTrain_0205/samplevectors_internal/{pathogen}/filenames_test.pt')
    rows_test = []
    for filename in filenames_internal_test:
        # dataframe row to list
        row = clinicals.loc[clinicals['Filename'] == filename.replace(filename.split('_')[0] + '_' + filename.split('_')[1] + '_', '')]
        row = row[feature_columns].values.tolist()
        rows_test += row

    all_rawvecs = torch.tensor(rows, dtype=torch.float32)
    all_rawvecs_test = torch.tensor(rows_test, dtype=torch.float32)

    torch.save(all_rawvecs, f'/home/yhchoi/FSTrain_0205/samplevectors_internal/{pathogen}/clinical_raw_featurevectors_train.pt')
    torch.save(all_rawvecs_test, f'/home/yhchoi/FSTrain_0205/samplevectors_internal/{pathogen}/clinical_raw_featurevectors_test.pt')

In [28]:
internal_minmax = pd.read_excel('/home/yhchoi/FSTrain_0205/extdata_labels/sinchon_minmax.xlsx')
internal_min_dict = internal_minmax.iloc[0].to_dict()
internal_max_dict = internal_minmax.iloc[1].to_dict()
clinicals_ext = pd.read_csv('/home/yhchoi/24BMTesting_ExtData/clinical/external_clinical_data.csv')
clinicals_ext.fillna(-1, inplace=True)

# modify every 'filename' in clinicals
clinicals_ext['Filename'] = clinicals_ext['Filename'].apply(lambda x: x.replace(x.split('_')[0] + '_', '', 1))

columns = list(clinicals_ext.columns)
columns[2] = 'Pathogen'
columns[3] = 'Prognosis'
clinicals_ext.columns = columns
columns_to_exclude = columns[:4]

no_minmax = []
def minmax_scale(column):
    if column.name in columns_to_exclude:
        return column
    try:
        return (column - internal_min_dict[column.name]) / (internal_max_dict[column.name] - internal_min_dict[column.name])
    except:
        no_minmax.append(column.name)
        print(f'No minmax for {column.name}')
        return column

scaled_df = clinicals_ext.apply(minmax_scale)
scaled_df.fillna(-1, inplace=True)

for pathogen in [0, 1, 2, 3]:
        feature_columns = acfdict_bykey[1] + acfdict_bykey[3] + acfdict_bykey[4] + acfdict_bykey[5] + acfdict_bykey[6] + ['CXR_abnormal', 'CT_abnormal']
        filenames_internal_test = torch.load(f'/home/yhchoi/FSTrain_0205/samplevectors/{pathogen}/filenames_test.pt')
        rows_test = []
        for filename in filenames_internal_test:
            # dataframe row to list
            row = clinicals_ext.loc[clinicals_ext['Filename'] == filename.replace(filename.split('_')[0] + '_' + filename.split('_')[1] + '_', '')]
            row = row[feature_columns].values.tolist()
            if row == []:
                row = clinicals.loc[clinicals['Filename'] == filename.replace(filename.split('_')[0] + '_' + filename.split('_')[1] + '_', '')]
                row = row[feature_columns].values.tolist()
            if row == []:
                row = clinicals_ext.loc[clinicals_ext['Filename'] == '11281775_20230801114702_1201_3d_t1_tfe_sag_f_gd']
                row = row[feature_columns].values.tolist()
            rows_test += row

        all_rawvecs_test = torch.tensor(rows_test, dtype=torch.float32)

        torch.save(all_rawvecs_test, f'/home/yhchoi/FSTrain_0205/samplevectors/{pathogen}/clinical_raw_featurevectors_test.pt')

feature_columns = acfdict_bykey[1] + acfdict_bykey[3] + acfdict_bykey[4] + acfdict_bykey[5] + acfdict_bykey[6]
torch.save(feature_columns, '/home/yhchoi/FSTrain_0205/samplevectors_internal/feature_names.pt')

No minmax for CSF_cryptoccocal
No minmax for Serum_cryptococcal
No minmax for CSF_HSV_IgM
No minmax for CSF_VZV_IgM
No minmax for Serum_HIV


In [19]:
clinicals

,Filename,ID,label,mRS_at_discharge,Mentality_abnormal,sex,TB,seizure,CT_abnormal,CXR_abnormal,...,serum_CRP,serum_ESR,serum_procalcitonin,serum_lactate,CCI,CSF_cryptoccocal,Serum_cryptococcal,CSF_HSV_IgM,CSF_VZV_IgM,Serum_HIV
0,10291451_10291451_1001_3d_t1_tfe_sag_f_gd,10291451,0,3,0,2,0,0,1.0,0.0,...,0.024006,0.067797,0.000000,-1.000000,0.333333,0.0,0.0,0.0,0.0,0.0
1,10335105_1001_3d_t1_tfe_sag_f_gd,10335105,0,2,0,1,0,0,-1.0,0.0,...,0.007202,0.474576,0.000121,-1.000000,0.111111,0.0,0.0,0.0,0.0,-1.0
2,10335105_901_3d_t1_bb_imsde_sag_f_gdcompo,10335105,0,2,0,1,0,0,-1.0,0.0,...,0.007202,0.474576,0.000121,-1.000000,0.111111,0.0,0.0,0.0,0.0,-1.0
3,10335105_2001_3d_t1_tfe_sag_f_gd,10335105,0,2,0,1,0,0,-1.0,0.0,...,0.007202,0.474576,0.000121,-1.000000,0.111111,0.0,0.0,0.0,0.0,-1.0
4,10335105_901_3d_t1_tfe_sag_f_gd,10335105,0,2,0,1,0,0,-1.0,0.0,...,0.007202,0.474576,0.000121,-1.000000,0.111111,0.0,0.0,0.0,0.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,9908140_9908140_801_3d_t1_tfe_sag_f_gd,9908140,3,1,0,2,0,0,0.0,0.0,...,-1.000000,0.533898,0.000000,-1.000000,0.111111,0.0,0.0,0.0,0.0,0.0
446,9915963_1301_3d_t1_tfe_sag_f_gd,9915963,3,0,1,1,0,1,0.0,0.0,...,0.039210,0.101695,0.000485,0.067797,0.000000,0.0,0.0,0.0,0.0,0.0
447,9929551_1201_3d_t1_tfe_sag_f_gd,9929551,3,0,0,2,0,0,0.0,0.0,...,0.024273,0.779661,0.000000,0.042373,0.000000,0.0,0.0,0.0,0.0,0.0
448,9940380_1601_3d_t1_tfe_sag_f_gd,9940380,3,0,0,2,0,0,-1.0,-1.0,...,0.042945,0.559322,-1.000000,-1.000000,0.000000,0.0,0.0,0.0,0.0,0.0
